In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### enrichment for upstream TEs

In [2]:
#open p value table
df = pd.read_excel('<upstream_p_values.xlsx>')
df = df.sort_values(by = 'p')
matrix = pd.read_excel('<upstream_TE_association_transposed_matrix_from_04.xlsx>', index_col=0)

second = []
for i in df['NAME']:
    second.append('_'.join(i.split('.')[:2])[1:])
df['second'] = second

In [3]:
#split for alpha level below/above .05
alpha = 0.05
df_sig = df[df['p']<= alpha] 
df_nonsig = df[df['p'] > alpha] 

In [4]:
dic_sig = {}
for i in set(df_nonsig['second']):
    dic_sig[i] = list(df_sig['second']).count(i)
    
dic_nonsig = {}
for i in set(df_nonsig['second']):
    dic_nonsig[i] = list(df_nonsig['second']).count(i)

In [5]:
df_bins = pd.DataFrame()
df_bins['bins'] = list(dic_nonsig.keys())
df_bins['sig'] = list(dic_sig.values())
df_bins['not_sig'] = list(dic_nonsig.values())


In [6]:
second = []
for i in df['NAME']:
    second.append('_'.join(i.split('.')[:2])[1:])
df['second'] = second

In [7]:
df_p = df[df['p'] <= 0.05]

In [8]:
df_bins

,bins,sig,not_sig
0,DNA damage response_DNA-protein crosslink (DPC...,0,1
1,External stimuli response_gravity,0,9
2,Coenzyme metabolism_S-adenosyl methionine (SAM...,0,1
3,Cell wall organisation_callose,0,7
4,Lipid metabolism_lipid droplet-associated acti...,1,14
...,...,...,...
208,External stimuli response_pathogen,0,68
209,Protein modification_cysteine disulfide formation,0,10
210,Coenzyme metabolism_tetrapyrrole biosynthesis,0,12
211,Carbohydrate metabolism_fermentation,0,3


In [9]:
#delete small groups
for i in df_bins.index:
    if (int(df_bins.loc[i]['sig']) + int(df_bins.loc[i]['not_sig'])) < 10: 
        df_bins = df_bins.drop(index = i)
df_bins

,bins,sig,not_sig
4,Lipid metabolism_lipid droplet-associated acti...,1,14
5,Photosynthesis_calvin cycle,1,16
6,Protein translocation_endoplasmic reticulum,0,19
8,RNA biosynthesis_RNA polymerase II-dependent t...,2,68
9,Cell division_meiotic recombination,1,37
...,...,...,...
205,RNA biosynthesis_transcriptional regulation,20,827
206,Cytoskeleton organisation_microtubular network,1,52
208,External stimuli response_pathogen,0,68
209,Protein modification_cysteine disulfide formation,0,10


In [10]:
import scipy.stats as stats
p_values = []
odds = []
for code in df_bins['bins']:
    oddsratio, pvalue = stats.fisher_exact([[sum(df_bins['sig'][df_bins['bins'] == code]),
                                             sum(df_bins['sig']) - sum(df_bins['sig'][df_bins['bins'] == code])],
                                            [sum(df_bins['not_sig'][df_bins['bins'] == code]), 
                                             sum(df_bins['not_sig']) - sum(df_bins['not_sig'][df_bins['bins'] == code])]], alternative='two-sided')  
    p_values.append(pvalue)
    odds.append(oddsratio)
df_bins['p_values'] = p_values
df_bins['odds_ratio'] = odds
df_bins = df_bins.sort_values(by = 'p_values')
#df_bins[df_bins['p_values'] <= 0.05]
df_bins[:20]

,bins,sig,not_sig,p_values,odds_ratio
90,Photosynthesis_photorespiration,3,8,0.000878,20.526316
103,not assigned_not annotated',40,3132,0.005013,0.617482
10,Protein biosynthesis_ribosome biogenesis,7,122,0.009182,3.170012
162,Protein homeostasis_protein quality control,3,36,0.033524,4.550239
132,Protein homeostasis_autophagy,2,21,0.064839,5.182313
20,Protein homeostasis_ubiquitin-proteasome system,10,304,0.081239,1.814747
70,Protein modification_glycosylation,2,25,0.085851,4.351619
201,RNA processing_RNA modification,3,55,0.088446,2.973380
164,Cell wall organisation_cutin and suberin,2,35,0.145143,3.105578
72,Cell division_cytokinesis,2,35,0.145143,3.105578
